In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import settings
import torch
from ollama import chat, ChatResponse


c:\Users\willb\Documents\Projetos\DataThon-Fiap\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from huggingface_hub import login
login(token=settings.login)

In [3]:
# Load model directly

#google/gemma-3-1b-it
#meta-llama/Llama-3.1-8B-Instruct
#Qwen/Qwen3-Next-80B-A3B-Instruct
#deepseek-ai/DeepSeek-R1-0528-Qwen3-8B
model_name = "google/gemma-3-1b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          cache_dir=f"../models/{model_name}",
                                          load_in_8bit=True,
                                          dtype="float16"
                                          )

model = AutoModelForCausalLM.from_pretrained(model_name
                                             ,device_map="auto"
                                             ,offload_folder="../offload"
                                             )


In [4]:
# Base de conhecimento (exemplo simples)
curriculo = '''Sou analista de dados, bacharel em Sistemas de Informação e atualmente curso uma
 pós-graduação em Engenharia de Machine Learning. Meu foco está na análise e
 engenharia de dados, além da automação de processos, sempre buscando
 transformar dados complexos em informações estratégicas e garantir soluções
 eficientes. Combino criatividade e pensamento analítico com senso de urgência,
 proatividade e trabalho em equipe para enfrentar desafios com foco e resiliência,
 enquanto aprimoro constantemente minhas habilidades técnicas e de liderança.
 
 No meu cargo atual, desenvolvo scripts em Python para criar sistemas internos, extrair
 dados via web scraping e automatizar tarefas rotineiras. Além disso, trabalho com
 SQL para manipulação eficiente de dados em um ambiente de Big Data, sempre
 buscando otimizar consultas. Com os dados refinados, crio painéis interativos e
 visualizações no Power BI e outras ferramentas, transformando conjuntos de dados
 complexos em insights acionáveis que auxiliam na tomada de decisões em diversas
 áreas do negócio.
 
 Python (Avançado)
 SQL (Avançado)
 Sistemas de Data Viz
 Inglês Fluente
 Metodologias Ágeis
 Docker
 Spark
 Algoritmos de
 Modelagem'''


In [5]:
job_description = '''Job Description

Perfil de um Desenvolvedor Python OCI para migrar um Data Lake da AWS para o OCI

Desenvolvedor sênior de Python
Uso de SDKs (OCI SDK em Python)
Experiência no tratamento de dados nos formatos Parquet e JSON
Otimização de jobs Spark a nível de código (partições, joins, cache)


Serviços OCI

OCI Object Storage
OCI Data Flow
OCI SQL Endpoint
OCI SDK para Python
OCI Function
OCI ADW


Responsibilities

Descrição das atividades:

Gerenciamento de acessos a buckets/objetos.
Automação de deployments.
Desenvolvimento de scripts para copiar dados entre AWS e OCI (upload/download multipart, paralelismo).
Migração de códigos AWS para OCI.
Configuração do Data Flow.
Consultas em SQL Endpoint.
Validações pós-migração.'''

In [6]:
def teste_rag(curriculo, job_description):

	prompt = f"""
    Contexto:
    Sua tarefa é ler o texto do currículo de um candidato e a descrição da vaga e gerar exatamente:

    1. Cinco pontos de destaque do candidato, em frases curtas e diretas (máximo 2 linhas cada), resumindo experiências, conquistas, habilidades e competências mais relevantes para a vaga.
    2. Cinco perguntas estratégicas para entrevista, que explorem motivação, comportamento, soft skills, competências ou requisitos da vaga que não aparecem claramente no currículo.

    Instruções obrigatórias (siga à risca):
    - Use apenas informações do currículo e da vaga. Não invente fatos.
    - Seja objetivo, claro e direto nos pontos de destaque (formato de mini resumo).
    - As perguntas devem ser diferentes entre si, relevantes para a vaga e incentivar respostas com exemplos concretos.
    - Nenhum ponto de destaque pode ser repetido.
    - Nenhuma pergunta pode ser repetida.
    - Nenhuma pergunta pode repetir informações já abordadas nos pontos de destaque.
    - Considere experiências passadas, formação acadêmica, habilidades técnicas e comportamentais.
    - Não retorne o texto de entrada, nem o contexto, nem explicações adicionais.

    - A saída deve estar **exclusivamente em formato JSON seguindo o template abaixo, sem qualquer texto fora dele:

    ```json
      "pontos_de_destaque": [
        "Resumo curto do ponto 1",
        "Resumo curto do ponto 2",
        "Resumo curto do ponto 3",
        "Resumo curto do ponto 4",
        "Resumo curto do ponto 5"
      ],
      "perguntas_entrevista": [
        "Pergunta 1",
        "Pergunta 2",
        "Pergunta 3",
        "Pergunta 4",
        "Pergunta 5"
      ]
    ```
      
    Entrada: 

    Currículo:
    {curriculo}

    Vaga:
    {job_description}

    """

	messages = [
		{"role": "system", "content": "Você é um assistente de recrutamento virtual."},
        {"role": "user", "content": prompt}
	]
	inputs = tokenizer.apply_chat_template(
		messages,
		add_generation_prompt=True,
		tokenize=True,
		return_dict=True,
		return_tensors="pt",
	).to(model.device)

	outputs = model.generate(**inputs, max_new_tokens=500)
	return tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])

In [7]:
teste = teste_rag(curriculo, job_description)

In [8]:
teste 

'```json\n{\n  "pontos_de_destaque": [\n    "Analista de dados com formação em Sistemas de Informação e uma pós-graduação em Engenharia de Machine Learning, com foco em análise e engenharia de dados, automação de processos e soluções eficientes.",\n    "Experiência comprovada em desenvolvimento de scripts Python para criação de sistemas internos, extração de dados via web scraping e automatização de tarefas rotineiras.",\n    "Domínio de SQL para manipulação eficiente de dados em ambientes de Big Data e otimização de consultas através de SQL.",\n    "Habilidade em trabalhar com frameworks de visualização de dados como Power BI e criar painéis interativos e visualizações.",\n    "Proficiência em metodologias ágeis e conhecimento em Docker e Spark para otimizar o desenvolvimento e escalabilidade de soluções."\n  ],\n  "perguntas_entrevista": [\n    "Considerando a migração de um Data Lake da AWS para o OCI, quais são os principais desafios que você identificaria e como você os abordaria?

In [15]:
import json

In [16]:
# Converter a string para um dicionário Python
dados = json.loads(teste)

# Acessando os pontos de destaque
print("Pontos de destaque:")
for ponto in dados['pontos_de_destaque']:
    print("-", ponto)

# Acessando as perguntas de entrevista
print("\nPerguntas de entrevista:")
for pergunta in dados['perguntas_entrevista']:
    print("-", pergunta)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)